# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [199]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [200]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from itertools import combinations

In [201]:
def build_classifiers():
    # fill this part
    
    #Linear Regression
    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)

    #Nearest Neighbours
    nearest_neighbors = KNeighborsClassifier()
    nearest_neighbors.fit(data_set, labels)

    #Linear SVM
    linear_svc = SVC()
    linear_svc.fit(data_set, labels)
    
    #Decision Tree
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(data_set, labels)
 
    #Naive Bayes
    naive_bayes = GaussianNB()
    naive_bayes.fit(data_set, labels)
    
    #QDA
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)
    
    return linear_regression, nearest_neighbors, linear_svc, decision_tree, naive_bayes, qda  # and here

In [202]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier() # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [203]:
classifiers = build_classifiers()

#combine
combination = combinations(classifiers, 3)
best_classifier = None
best_accuracy = 0

for classifiers in combination:
    predicted = build_stacked_classifier(classifiers)
    accuracy = accuracy_score(test_labels, predicted)
    if accuracy > best_accuracy:
        best_classifier = classifiers
        best_accuracy = accuracy
    print(classifiers, accuracy)

print("\n############################################################################")
print("Best accuracy =", best_accuracy, "for", best_classifier)

(LinearRegression(), KNeighborsClassifier(), SVC()) 0.1
(LinearRegression(), KNeighborsClassifier(), DecisionTreeClassifier()) 0.75
(LinearRegression(), KNeighborsClassifier(), GaussianNB()) 0.7
(LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) 0.75
(LinearRegression(), SVC(), DecisionTreeClassifier()) 0.6
(LinearRegression(), SVC(), GaussianNB()) 0.6
(LinearRegression(), SVC(), QuadraticDiscriminantAnalysis()) 0.6
(LinearRegression(), DecisionTreeClassifier(), GaussianNB()) 0.1
(LinearRegression(), DecisionTreeClassifier(), QuadraticDiscriminantAnalysis()) 0.75
(LinearRegression(), GaussianNB(), QuadraticDiscriminantAnalysis()) 0.6
(KNeighborsClassifier(), SVC(), DecisionTreeClassifier()) 0.85
(KNeighborsClassifier(), SVC(), GaussianNB()) 0.85
(KNeighborsClassifier(), SVC(), QuadraticDiscriminantAnalysis()) 0.85
(KNeighborsClassifier(), DecisionTreeClassifier(), GaussianNB()) 0.0
(KNeighborsClassifier(), DecisionTreeClassifier(), QuadraticDiscriminantAnalys

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [204]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

Fill the two functions below:

In [205]:
def set_new_weights(model):
    # fill the code here (two lines)
    I_1 = np.add(1, calculate_accuracy_vector(model.predict(test_set), test_labels))
    w = I_1/np.sum(I_1)
    
    return w

Train the classifier with the code below:

In [206]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00131579 0.00131579 0.00131579 0.00131579 0.00065789 0.00131579
 0.00065789 0.00131579 0.00131579 0.00065789 0.00065789 0.00131579
 0.00131579 0.00065789 0.00065789 0.00131579 0.00131579 0.00131579
 0.00131579 0.00131579 0.00131579 0.00065789 0.00065789 0.00131579
 0.00065789 0.00065789 0.00131579 0.00131579 0.00065789 0.00065789
 0.00065789 0.00065789 0.00131579 0.00131579 0.00065789 0.00065789
 0.00065789 0.00065789 0.00065789 0.00131579 0.00131579 0.00065789
 0.00065789 0.00131579 0.00065789 0.00065789 0.00131579 0.00131579
 0.00131579 0.00131579 0.00131579 0.00065789 0.00065789 0.00065789
 0.00131579 0.00131579 0.00131579 0.00131579 0.00065789 0.00065789
 0.00131579 0.00065789 0.00131579 0.00065789 0.00131579 0.00131579
 0.00065789 0.00131579 0.00131579 0.00065789 0.00131579 0.00065789
 0.00065789 0.00065789 0.00065789 0.00065789 0.00131579 0.00131579
 0.00065789 0.00131579 0.00131579 0.00065789 0.00065789 0.00131579
 0.00065789 0.00131579 0.00065789 0.00131579 0.00065789 0.0013

Set the validation data set:

In [207]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [210]:
def get_prediction(x):
    # fill the code here (5-6 lines)
    predictions = []
    
    for i in range(len(classifiers)):
        predicted = classifiers[i].predict(x)
        predictions.append(predicted)
    counter = [0.0, 0.0]
    
    for i, prediction in enumerate(predictions):
        if prediction == 0:
            counter[0] =+ weights[i]
        else:
            counter[1] =+ weights[i]

    return int(counter[0]<counter[1])

Test it:

In [211]:
prediction = get_prediction(validate_x)

print(prediction)

0
